In [2]:
# Importar os módulos necessários do PySpark e do PyArrow
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pyarrow as pa
import pyarrow.dataset as ds

# Iniciar uma sessão Spark
spark = SparkSession.builder \
    .appName("Online food ordering") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.3.1") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.ic.catalog-impl", "org.apache.iceberg.jdbc.JdbcCatalog") \
    .config("spark.sql.catalog.ic", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.ic.uri", "jdbc:postgresql://postgres:5432/iceberg_db") \
    .config("spark.sql.catalog.ic.password", "postgres") \
    .config("spark.sql.catalog.ic.user", "postgres") \
    .getOrCreate()

# Definir o esquema para o DataFrame
schema = StructType([
    StructField("Age", IntegerType(), True),
    StructField("Gender", StringType(), True),
    StructField("Marital Status", StringType(), True),
    StructField("Occupation", StringType(), True),
    StructField("Monthly Income", StringType(), True),
    StructField("Educational Qualifications", StringType(), True),
    StructField("Family size", IntegerType(), True),
    StructField("latitude", StringType(), True),
    StructField("longitude", StringType(), True),
    StructField("Pin code", StringType(), True),
    StructField("Output", StringType(), True),
    StructField("Feedback", StringType(), True)
])

# Carregar o arquivo CSV para um DataFrame PySpark
df = spark.read.csv("onlinefoods.csv", schema=schema, header=True)

# Criar uma tabela Iceberg no PostgreSQL
df.write.format("iceberg") \
    .mode("overwrite") \
    .option("path", "iceberg_db") \
    .option("createTableColumnTypes", "Age INT, Gender STRING, `Marital Status` STRING, Occupation STRING, `Monthly Income` STRING, `Educational Qualifications` STRING, `Family size` INT, latitude STRING, longitude STRING, `Pin code` STRING, Output STRING, Feedback STRING") \
    .save()

# Criar um dataset Iceberg a partir do caminho de armazenamento no PostgreSQL
dataset = ds.dataset("jdbc:postgresql://postgres:5432/iceberg_db", format="iceberg")

# Exibir as informações da tabela Iceberg
print(dataset.schema)

# Parar a sessão Spark
spark.stop()

Py4JJavaError: An error occurred while calling o82.save.
: org.apache.iceberg.hive.RuntimeMetaException: Failed to connect to Hive Metastore
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:84)
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:34)
	at org.apache.iceberg.ClientPoolImpl.get(ClientPoolImpl.java:125)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:56)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:51)
	at org.apache.iceberg.hive.CachedClientPool.run(CachedClientPool.java:122)
	at org.apache.iceberg.hive.HiveTableOperations.doRefresh(HiveTableOperations.java:158)
	at org.apache.iceberg.BaseMetastoreTableOperations.refresh(BaseMetastoreTableOperations.java:97)
	at org.apache.iceberg.BaseMetastoreTableOperations.current(BaseMetastoreTableOperations.java:80)
	at org.apache.iceberg.BaseMetastoreCatalog.loadTable(BaseMetastoreCatalog.java:47)
	at org.apache.iceberg.spark.SparkCatalog.load(SparkCatalog.ja